## ChatGLM2 推理

### 配置实验环境
The following code is copied from baichuan_infer.ipynb

In [2]:
from _common import *
from transformers import TextStreamer
device_ids = [0, 1]
logger.info(device_ids)
select_device(device_ids)

[2023-07-02 21:48:47,527] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2023-07-02 21:48:48,006 - modelscope - INFO - PyTorch version 2.0.1 Found.
2023-07-02 21:48:48,007 - modelscope - INFO - Loading ast index from /home/hackathon/.cache/modelscope/ast_indexer
2023-07-02 21:48:48,032 - modelscope - INFO - Loading done! Current index file version is 1.6.2, with md5 ddf811ee982377c1357284a2bfda3dec and a total number of 861 components indexed
2023-07-02 21:48:48,708 - modelscope - INFO - [0, 1]
2023-07-02 21:48:48,848 - modelscope - INFO - Using device: cuda:0,1


device(type='cuda', index=0)

### 导入Model, Tokenizer
Note: 你需要设置CKPT_FPATH的内容, 指向`.bin`文件, 或`.pth`文件

In [3]:
CKPT_FAPTH = '/home/hackathon/my_git/agent/runs/chatglm2/v1-20230702-203505/output_best/pytorch_model.bin'
LORA_TARGET_MODULES = ['query_key_value']

model, tokenizer = get_chatglm2_model_tokenizer()
if tokenizer.eos_token_id is None:
    tokenizer.eos_token_id = tokenizer.pad_token_id
if tokenizer.bos_token_id is None:
    tokenizer.bos_token_id = 1
model.bfloat16()  # Consistent with training

2023-07-02 21:48:49,227 - modelscope - INFO - Development mode use revision: v1.0.3
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'ChatGLMTokenizer'. 
The class this function is called from is 'ChatGLM2Tokenizer'.
2023-07-02 21:48:49,572 - modelscope - INFO - initialize model from /home/hackathon/.cache/modelscope/hub/ZhipuAI/chatglm2-6b
Failed to load cpm_kernels:No module named 'cpm_kernels'
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

ChatGLM2ForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

### 导入Lora
The following code is copied from baichuan_infer.ipynb

In [4]:
LORA_RANK = 8
LORA_ALPHA = 32
LORA_DROPOUT_P = 0  # Arbitrary value
lora_config = LoRAConfig(
    replace_modules=LORA_TARGET_MODULES,
    rank=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT_P,
    pretrained_weights=CKPT_FAPTH)
logger.info(f'lora_config: {lora_config}')
Swift.prepare_model(model, lora_config)

2023-07-02 21:48:56,970 - modelscope - INFO - lora_config: LoRAConfig(rank=8, replace_modules=['query_key_value'], lora_alpha=32, lora_dropout=0, merge_weights=True, use_merged_linear=False, enable_lora=None, fan_in_fan_out=False, bias='none', only_lora_trainable=True, pretrained_weights='/home/hackathon/my_git/agent/runs/chatglm2/v1-20230702-203505/output_best/pytorch_model.bin')


ChatGLM2ForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_layer): Linear(in

### 导入Dataset
The following code is copied from baichuan_infer.ipynb

In [5]:
test_dataset = make_dataset('validation', lambda system, user, assistant:
                            {'system': system, 'user': user, 'assistant': assistant})

2023-07-02 21:49:01,924 - modelscope - INFO - No subset_name specified, defaulting to the default
2023-07-02 21:49:02,374 - modelscope - WARNING - Reusing dataset ms_hackathon_23_agent_train_dev (/home/hackathon/.cache/modelscope/hub/datasets/modelscope/ms_hackathon_23_agent_train_dev/master/data_files)
2023-07-02 21:49:02,375 - modelscope - INFO - Generating dataset ms_hackathon_23_agent_train_dev (/home/hackathon/.cache/modelscope/hub/datasets/modelscope/ms_hackathon_23_agent_train_dev/master/data_files)
2023-07-02 21:49:02,375 - modelscope - INFO - Reusing cached meta-data file: /home/hackathon/.cache/modelscope/hub/datasets/modelscope/ms_hackathon_23_agent_train_dev/master/data_files/941b733ec0354c2172a3386d8788bb37


Extracting data files: 0it [00:00, ?it/s]

100%|██████████| 285/285 [00:00<00:00, 1577014.04it/s]


### 推理
The following code is copied from baichuan_infer.ipynb

In [8]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
for d in test_dataset[:5]:
    system = d['system']
    user = d['user']
    assistant = d['assistant']
    input_ids = tokenize_function(system, user, None, tokenizer)['input_ids']
    print(f'[TEST]{tokenizer.decode(input_ids)}', end='')
    input_ids = torch.tensor(input_ids)[None].cuda()
    attention_mask = torch.ones_like(input_ids)
    generate_ids = model.generate(input_ids=input_ids, max_new_tokens=512,
                                  attention_mask=attention_mask,
                                  streamer=streamer, pad_token_id=tokenizer.pad_token_id, 
                                  temperature=0.7, top_k=50, do_sample=True)
    print()
    print(f'[LABELS]{assistant}')
    print('-----------------------------------------------------------------------------------')
    # input('next[ENTER]')

[TEST]你是达摩院的ModelScopeGPT（魔搭助手），你是个大语言模型， 是2023年达摩院的工程师训练得到的。你有多种能力，可以通过插件集成魔搭社区的模型api来回复用户的问题，还能解答用户使用模型遇到的问题和模型知识相关问答。1. {"plugin_name": "modelscope_speech-generation", "plugin_owner": "ModelScopeGPT", "plugin_type": "default", "plugin_schema_for_model": {"name": "modelscope_speech-generation", "description": "针对回复的内容，用语音表示，同时可以选择是男声或者女声", "url": "http://90.49.118.175:2603/", "paths": [{"name": "modelscope_speech-generation", "model_id": "/damo/speech_sambert-hifigan_tts_zh-cn_16k", "method": "post", "description": "针对回复的内容，用语音表示，同时可以选择是男声或者女声", "parameters": [{"name": "text", "description": "要转成语音的文本", "required": "True"}, {"name": "gender", "description": "用户身份", "required": "True"}]}]}}

2. {"plugin_name": "modelscope_speech-generation", "plugin_owner": "ModelScopeGPT", "plugin_type": "default", "plugin_schema_for_model": {"name": "modelscope_speech-generation", "description": "针对回复的内容，用语音表示，同时可以选择是男声或者女声", "url": "http://132.94.116.115:5983/", "paths": [{"name": "modelscope_speec